In [ ]:
#base
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size



In [ ]:
# Implementación de la capa densa (FCLayer)

class FCLayer:
    def __init__(self, input_size, output_size, lambda_reg=0.0):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.lambda_reg = lambda_reg

    def forward(self, input):
        self.input = input
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward(self, output_gradient, learning_rate):
        input_gradient = np.dot(output_gradient, self.weights.T)
        weights_gradient = np.dot(self.input.T, output_gradient)
        weights_gradient += self.lambda_reg * self.weights
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * np.mean(output_gradient, axis=0)
        return input_gradient



In [ ]:
# Implementación de la capa de activación
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        self.output = self.activation(self.input)
        return self.output

    def backward(self, output_gradient, learning_rate):
        return output_gradient * self.activation_prime(self.input)



In [ ]:
# Implementación de la capa de pooling
class PoolingLayer:
    def __init__(self, pool_size, mode='max'):
        self.pool_size = pool_size
        self.mode = mode

    def forward(self, input):
        self.input = input
        self.output = np.zeros((input.shape[0], input.shape[1] // self.pool_size))
        for i in range(0, input.shape[1], self.pool_size):
            if self.mode == 'max':
                self.output[:, i // self.pool_size] = np.max(input[:, i:i + self.pool_size], axis=1)
            elif self.mode == 'average':
                self.output[:, i // self.pool_size] = np.mean(input[:, i:i + self.pool_size], axis=1)
        return self.output

    def backward(self, output_gradient, learning_rate):
        input_gradient = np.zeros_like(self.input)
        for i in range(0, self.input.shape[1], self.pool_size):
            if self.mode == 'max':
                max_indices = np.argmax(self.input[:, i:i + self.pool_size], axis=1)
                input_gradient[:, i:i + self.pool_size] = (np.arange(self.pool_size) == max_indices[:, None]) * output_gradient[:, i // self.pool_size][:, None]
            elif self.mode == 'average':
                input_gradient[:, i:i + self.pool_size] = output_gradient[:, i // self.pool_size][:, None] / self.pool_size
        return input_gradient



In [ ]:
# Implementación de la red neuronal
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward(output)
            result.append(output)
        return result

    def fit(self, X_train, y_train, epochs, learning_rate):
        samples = len(X_train)
        for epoch in range(epochs):
            print(f'Epoch {epoch+1}/{epochs}')
            for i in range(samples):
                output = X_train[i]
                for layer in self.layers:
                    output = layer.forward(output)
                loss = self.loss(y_train[i], output)
                output_gradient = self.loss_prime(y_train[i], output)
                for layer in reversed(self.layers):
                    output_gradient = layer.backward(output_gradient, learning_rate)
            print(f'Loss: {loss}')



In [ ]:
# cargado de datos
digits = load_digits()
X = digits.data
y = digits.target
y_one_hot = np.zeros((y.size, 10))
y_one_hot[np.arange(y.size), y] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.3, random_state=42)

#creacion de la red
entrada_dim = X_train.shape[1]
model = Network()
lambda_reg = 0.001
model.add(FCLayer(entrada_dim, 256, lambda_reg=lambda_reg))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(PoolingLayer(pool_size=2, mode='max'))
model.add(FCLayer(128, 256, lambda_reg=lambda_reg))
model.add(ActivationLayer(sigmoid, sigmoid_prime))
model.add(FCLayer(256, 10, lambda_reg=lambda_reg))
model.add(ActivationLayer(sigmoid, sigmoid_prime))

model.use(mse, mse_prime)
model.fit(X_train, y_train, epochs=20, learning_rate=0.1)

y_hat = model.predict(X_test)
y_hat = [np.argmax(i) for i in y_hat]
y_test_labels = [np.argmax(i) for i in y_test]

print('Matriz de confusión:')
print(confusion_matrix(y_test_labels, y_hat))
print(f'Exactitud del modelo: {accuracy_score(y_test_labels, y_hat)}')
